##### Copyright 2018 Los autores de TensorFlow.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Using the SavedModel format

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/saved_model"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/saved_model.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/saved_model.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/saved_model.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar libreta</a></td>
</table>

Un modelo guardado contiene un programa TensorFlow completo, incluidos pesos y cálculos. No requiere el código de creación del modelo original para ejecutarse, lo que lo hace útil para compartir o implementar (con [TFLite](https://tensorflow.org/lite) , [TensorFlow.js](https://js.tensorflow.org/) , [TensorFlow Serving](https://www.tensorflow.org/tfx/serving/tutorials/Serving_REST_simple) o [TensorFlow Hub](https://tensorflow.org/hub) ).

Este documento se sumerge en algunos de los detalles de cómo usar la API `tf.saved_model` de bajo nivel:

- Si está utilizando un `tf.keras.Model` , el `keras.Model.save(output_path)` puede ser todo lo que necesita: vea [Keras guardar y serializar](keras/save_and_serialize.ipynb)

- If you just want to save/load weights during training see the [guide to training checkpoints](./checkpoint.ipynb).


In [0]:
import os
import tempfile

from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf

tmpdir = tempfile.mkdtemp()

In [0]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [0]:
file = tf.keras.utils.get_file(
    "grace_hopper.jpg",
    "https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg")
img = tf.keras.preprocessing.image.load_img(file, target_size=[224, 224])
plt.imshow(img)
plt.axis('off')
x = tf.keras.preprocessing.image.img_to_array(img)
x = tf.keras.applications.mobilenet.preprocess_input(
    x[tf.newaxis,...])

<a id="saved_model_cli"></a>

## Detalles de la interfaz de línea de comandos de SavedModel

Puede utilizar la interfaz de línea de comandos (CLI) del modelo guardado para inspeccionar y ejecutar un modelo guardado. Por ejemplo, puede usar la CLI para inspeccionar los `SignatureDef` del modelo. La CLI le permite confirmar rápidamente que el tipo y la forma del tensor de entrada coinciden con el modelo. Además, si desea probar su modelo, puede usar la CLI para realizar una verificación de cordura al pasar entradas de muestra en varios formatos (por ejemplo, expresiones de Python) y luego obtener la salida.

### Instale la CLI del modelo guardado

En términos generales, puede instalar TensorFlow de cualquiera de las dos formas siguientes:

- Al instalar un binario de TensorFlow preconstruido.
- Construyendo TensorFlow a partir del código fuente.

Si instaló TensorFlow a través de un binario de TensorFlow prediseñado, entonces la CLI del modelo guardado ya está instalada en su sistema en el nombre de ruta `bin/saved_model_cli` .

Si compilaste TensorFlow a partir del código fuente, debes ejecutar el siguiente comando adicional para compilar `saved_model_cli` :

```
$ bazel build tensorflow/python/tools:saved_model_cli
```

### Descripción general de los comandos

La CLI del modelo guardado admite los siguientes dos comandos en un modelo guardado:

- `show` , que muestra los cálculos disponibles de un modelo guardado.
- `run` , que ejecuta un cálculo desde un modelo guardado.

### `show` comando

Un modelo guardado contiene una o más variantes de modelo (técnicamente, `v1.MetaGraphDef` s), identificadas por sus conjuntos de etiquetas. Para servir un modelo, es posible que se pregunte qué tipo de `SignatureDef` hay en cada variante del modelo y cuáles son sus entradas y salidas. El comando `show` le permite examinar el contenido del modelo guardado en orden jerárquico. Aquí está la sintaxis:

```
usage: saved_model_cli show [-h] --dir DIR [--all]
[--tag_set TAG_SET] [--signature_def SIGNATURE_DEF_KEY]
```

Por ejemplo, el siguiente comando muestra todos los conjuntos de etiquetas disponibles en el modelo guardado:

```
$ saved_model_cli show --dir /tmp/saved_model_dir
The given SavedModel contains the following tag-sets:
serve
serve, gpu
```

El siguiente comando muestra todas las claves `SignatureDef` disponibles para un conjunto de etiquetas:

```
$ saved_model_cli show --dir /tmp/saved_model_dir --tag_set serve
The given SavedModel `MetaGraphDef` contains `SignatureDefs` with the
following keys:
SignatureDef key: "classify_x2_to_y3"
SignatureDef key: "classify_x_to_y"
SignatureDef key: "regress_x2_to_y3"
SignatureDef key: "regress_x_to_y"
SignatureDef key: "regress_x_to_y2"
SignatureDef key: "serving_default"
```

Si hay *varias* etiquetas en el conjunto de etiquetas, debe especificar todas las etiquetas, cada etiqueta separada por una coma. Por ejemplo:

<pre>
$ saved_model_cli show --dir /tmp/saved_model_dir --tag_set serve,gpu
</pre>

Para mostrar todas las entradas y salidas de TensorInfo para un `SignatureDef` específico, pase la clave `SignatureDef` a la opción `signature_def` . Esto es muy útil cuando desea conocer el valor de la clave del tensor, el tipo y la forma de los tensores de entrada para ejecutar el gráfico de cálculo más adelante. Por ejemplo:

```
$ saved_model_cli show --dir \
/tmp/saved_model_dir --tag_set serve --signature_def serving_default
The given SavedModel SignatureDef contains the following input(s):
  inputs['x'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: x:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['y'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: y:0
Method name is: tensorflow/serving/predict
```

Para mostrar toda la información disponible en el modelo guardado, use la opción `--all` . Por ejemplo:

<pre>
$ saved_model_cli show --dir /tmp/saved_model_dir --all
MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['classify_x2_to_y3']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['inputs'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: x2:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['scores'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: y3:0
  Method name is: tensorflow/serving/classify

...

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['x'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: x:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['y'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: y:0
  Method name is: tensorflow/serving/predict
</pre>

### `run` comando

Invoque el comando de `run` para ejecutar un cálculo gráfico, pasando entradas y luego mostrando (y opcionalmente guardando) las salidas. Aquí está la sintaxis:

```
usage: saved_model_cli run [-h] --dir DIR --tag_set TAG_SET --signature_def
                           SIGNATURE_DEF_KEY [--inputs INPUTS]
                           [--input_exprs INPUT_EXPRS]
                           [--input_examples INPUT_EXAMPLES] [--outdir OUTDIR]
                           [--overwrite] [--tf_debug]
```

El comando `run` proporciona las siguientes tres formas de pasar entradas al modelo:

- La opción `--inputs` le permite pasar numpy ndarray en archivos.
- La opción `--input_exprs` le permite pasar expresiones de Python.
- La opción `--input_examples` le permite pasar `tf.train.Example` .

#### `--inputs`

Para pasar datos de entrada en archivos, especifique la opción `--inputs` , que toma el siguiente formato general:

```bsh
--inputs &lt;INPUTS&gt;
```

donde *ENTRADAS* es cualquiera de los siguientes formatos:

- `&lt;input_key&gt;=&lt;filename&gt;`
- `&lt;input_key&gt;=&lt;filename&gt;[&lt;variable_name&gt;]`

Puede pasar varias *ENTRADAS* . Si pasa varias entradas, use un punto y coma para separar cada una de las *ENTRADAS* .

`saved_model_cli` usa `numpy.load` para cargar el nombre de *archivo* . El nombre del *archivo* puede estar en cualquiera de los siguientes formatos:

- `.npy`
- `.npz`
- formato de pepinillo

Un archivo `.npy` siempre contiene un ndarray numpy. Por lo tanto, al cargar desde un archivo `.npy` , el contenido se asignará directamente al tensor de entrada especificado. Si especifica un *nombre_variable* con ese archivo `.npy` , el *nombre_variable* se ignorará y se emitirá una advertencia.

Al cargar desde un `.npz` (zip), puede especificar opcionalmente un nombre_variable para identificar la *variable* dentro del archivo zip para cargar la clave de tensor de entrada. Si no especifica un *nombre_variable* , la CLI del modelo guardado verificará que solo se incluye un archivo en el archivo zip y lo cargará para la clave de tensor de entrada especificada.

Al cargar desde un archivo pickle, si no se especifica ningún `variable_name` entre corchetes, lo que sea que esté dentro del archivo pickle se pasará a la clave de tensor de entrada especificada. De lo contrario, la CLI del modelo guardado supondrá que hay un diccionario almacenado en el archivo pickle y se usará el valor correspondiente a *variable_name* .

#### `--input_exprs`

Para pasar entradas a través de expresiones de Python, especifique la opción `--input_exprs` . Esto puede ser útil cuando no tiene archivos de datos por ahí, pero aún desea verificar la cordura del modelo con algunas entradas simples que coinciden con el tipo de d y la forma de los `SignatureDef` s del modelo. Por ejemplo:

```bsh
`&lt;input_key&gt;=[[1],[2],[3]]`
```

Además de las expresiones de Python, también puede pasar funciones numpy. Por ejemplo:

```bsh
`&lt;input_key&gt;=np.ones((32,32,3))`
```

(Tenga en cuenta que el módulo `numpy` ya está disponible para usted como `np` ).

#### `--input_examples`

Para pasar `tf.train.Example` como entradas, especifique la opción `--input_examples` . Para cada tecla de entrada, toma una lista de diccionarios, donde cada diccionario es una instancia de `tf.train.Example` . Las claves del diccionario son las funciones y los valores son las listas de valores para cada función. Por ejemplo:

```bsh
`&lt;input_key&gt;=[{"age":[22,24],"education":["BS","MS"]}]`
```

#### Guardar salida

De forma predeterminada, la CLI del modelo guardado escribe la salida en la salida estándar. Si se pasa un directorio a la opción `--outdir` , las salidas se guardarán como archivos `.npy` con el nombre de las claves de tensor de salida en el directorio dado.

Utilice `--overwrite` para sobrescribir los archivos de salida existentes.
